# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686610


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:06<01:36,  3.45s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<01:03,  2.45s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:45,  1.81s/it]

Rendering models:  20%|██        | 6/30 [00:07<00:31,  1.33s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:11<00:00,  4.28it/s]

Rendering models:  93%|█████████▎| 28/30 [00:11<00:00,  4.05it/s]

Rendering models:  97%|█████████▋| 29/30 [00:11<00:00,  4.56it/s]

Rendering models: 100%|██████████| 30/30 [00:11<00:00,  4.28it/s]

ElisabethB                            0.063681
not-logged-in-3e461b5a5635ce55ae0f    0.013941
JeremyR                               0.007851
not-logged-in-20a275b53abec8db9f41    0.126948
Almudys                               0.061492
shirogane                             0.000230
cschwefl                              0.000180
Staceyann116                          0.001627
not-logged-in-f5ac928c4dcd4e9cb619    0.001676
mkehrli                               0.006727
not-logged-in-f7e2260ed4a9517a3e4a    0.000894
not-logged-in-2908c5b97527b9f7279b    0.069347
Petrusperes                           0.015290
AntonKolev                            0.002571
Galaxy-one                            0.019768
redboy21                              0.008294
Roksolana                             0.000291
sn346808                              0.013269
sn345012                              0.011946
not-logged-in-3b41f5e8bd398117bfe6    0.001064
not-logged-in-3b540a42933c0240707e    0.067730
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())